In [2]:
from judo_footage_analysis.utils import get_spark

root = "/cs-share/pradalier/tmp/judo"

spark = get_spark(cores=4, mem="1g")
spark

24/04/04 19:01:04 WARN Utils: Your hostname, gtlpc106.georgiatech-metz.fr resolves to a loopback address: 127.0.1.1; using 192.93.8.106 instead (on interface enp0s31f6)
24/04/04 19:01:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/04 19:01:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/04 19:01:04 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [48]:
from pyspark.sql import functions as F

df = (
    spark.read.json(
        f"{root}/models/evaluation_embeddings_logistic_binary/v2/*/*/perf/*.json"
    )
    .withColumn("filename", F.input_file_name())
    .withColumn("feature", F.split(F.col("filename"), "/").getItem(10))
)
df.show()

+--------------------+-----------+-----------+--------------------+-------------------+-------------+---------+------------------+--------------+----------+------------------+--------------------+--------------------+
|         avg_metrics|      label|metric_name|         std_metrics|        test_metric|test_positive|test_size|         test_time|train_positive|train_size|        train_time|            filename|             feature|
+--------------------+-----------+-----------+--------------------+-------------------+-------------+---------+------------------+--------------+----------+------------------+--------------------+--------------------+
|[0.8372113080462782]|is_standing|         f1|[0.8372113080462782]|0.19610924055368498|            3|       90|0.8683209730079398|           103|       480|58.763832591008395|file:///cs-share/...|emb_entity_detect...|
|[0.8372113080462782]|is_standing|         f1|[0.8372113080462782]|0.19610924055368498|            3|       90|0.751439162995666

In [49]:
df.select(
    "label", "train_size", "train_positive", "test_size", "test_positive"
).distinct().show()

+-----------+----------+--------------+---------+-------------+
|      label|train_size|train_positive|test_size|test_positive|
+-----------+----------+--------------+---------+-------------+
|  is_active|       480|           238|       90|           45|
|is_standing|       480|           103|       90|            3|
|   is_match|       480|           378|       90|           60|
+-----------+----------+--------------+---------+-------------+



In [51]:
df.select(
    "label",
    "feature",
    F.round(F.col("avg_metrics")[0], 3).alias("avg_train_f1"),
    F.round("test_metric", 3).alias("test_f1"),
    F.col("train_time").cast("integer").alias("train_time"),
).orderBy("label", F.desc("test_f1")).show(truncate=False, n=100)

+-----------+-------------------------------+------------+-------+----------+
|label      |feature                        |avg_train_f1|test_f1|train_time|
+-----------+-------------------------------+------------+-------+----------+
|is_active  |emb_vanilla_yolov8n            |0.835       |0.575  |58        |
|is_active  |emb_entity_detection_v2        |0.835       |0.575  |63        |
|is_active  |emb_entity_detection_v2_dct_d64|0.783       |0.486  |60        |
|is_active  |emb_entity_detection_v2_dct_d32|0.802       |0.418  |58        |
|is_active  |emb_entity_detection_v2_dct_d16|0.746       |0.357  |45        |
|is_active  |emb_entity_detection_v2_dct_d8 |0.718       |0.283  |7         |
|is_active  |emb_vanilla_yolov8n_dct_d8     |0.718       |0.283  |6         |
|is_match   |emb_entity_detection_v2        |0.955       |0.582  |74        |
|is_match   |emb_entity_detection_v2_dct_d64|0.931       |0.582  |57        |
|is_match   |emb_entity_detection_v2_dct_d32|0.941       |0.582 